In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [4]:
# Load & clean places data
path = "/Users/remihendershott/GLP1-trends/data/PLACES__Local_Data_for_Better_Health,_Census_Tract_Data_2024_release_20251124.csv"

places_df = pd.read_csv(path)

places_df.head()

,Year,StateAbbr,StateDesc,CountyName,Category,Measure,Data_Value_Unit,Data_Value_Type,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,TotalPopulation,TotalPop18plus,LocationID,MeasureId,DataValueTypeID,Short_Question_Text
0,2022,FL,Florida,Miami-Dade,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,17.7,13.6,22.6,"1,434","1,005",12086009505,ACCESS2,CrdPrv,Health Insurance
1,2022,FL,Florida,Miami-Dade,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,12.0,9.5,15.4,"3,965","2,988",12086007806,ACCESS2,CrdPrv,Health Insurance
2,2022,FL,Florida,Miami-Dade,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,25.6,20.8,30.4,"3,042","2,298",12086003400,ACCESS2,CrdPrv,Health Insurance
3,2022,FL,Florida,Miami-Dade,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,33.6,28.4,38.9,"7,337","5,268",12086010804,ACCESS2,CrdPrv,Health Insurance
4,2022,FL,Florida,Miami-Dade,Prevention,Current lack of health insurance among adults ...,%,Crude prevalence,39.0,31.8,46.3,"2,535","2,156",12086005407,ACCESS2,CrdPrv,Health Insurance


In [8]:
columns_to_keep = [
    "Year",
    "StateAbbr",
    "StateDesc",
    "LocationID",
    "Category",
    "MeasureId",
    "Measure",
    "Data_Value",
    "Low_Confidence_Limit",
    "High_Confidence_Limit",
    "TotalPop18plus"
]

places_df_clean = places_df[columns_to_keep].copy()
places_df_clean.head()


,Year,StateAbbr,StateDesc,LocationID,Category,MeasureId,Measure,Data_Value,Low_Confidence_Limit,High_Confidence_Limit,TotalPop18plus
0,2022,FL,Florida,12086009505,Prevention,ACCESS2,Current lack of health insurance among adults ...,17.7,13.6,22.6,"1,005"
1,2022,FL,Florida,12086007806,Prevention,ACCESS2,Current lack of health insurance among adults ...,12.0,9.5,15.4,"2,988"
2,2022,FL,Florida,12086003400,Prevention,ACCESS2,Current lack of health insurance among adults ...,25.6,20.8,30.4,"2,298"
3,2022,FL,Florida,12086010804,Prevention,ACCESS2,Current lack of health insurance among adults ...,33.6,28.4,38.9,"5,268"
4,2022,FL,Florida,12086005407,Prevention,ACCESS2,Current lack of health insurance among adults ...,39.0,31.8,46.3,"2,156"


In [9]:
# Pivot wide
df_small = places_df_clean.copy()

df_wide = df_small.pivot_table(
    index=["LocationID", "StateAbbr", "StateDesc", "Year", "TotalPop18plus"],
    columns="MeasureId",
    values="Data_Value"
).reset_index()

# Flatten column names 
df_wide.columns.name = None

df_wide.head()


,LocationID,StateAbbr,StateDesc,Year,TotalPop18plus,ACCESS2,DIABETES,MHLTH,OBESITY
0,1001020100,AL,Alabama,2022,"1,370",11.2,13.8,20.0,39.3
1,1001020200,AL,Alabama,2022,"1,584",10.7,14.7,19.0,42.6
2,1001020300,AL,Alabama,2022,"2,485",10.9,14.1,19.7,40.3
3,1001020400,AL,Alabama,2022,"3,344",8.4,13.4,17.8,36.9
4,1001020501,AL,Alabama,2022,"3,369",7.9,11.4,16.4,35.6


In [ ]:
# Summary stats
df_wide[["OBESITY", "DIABETES", "MHLTH", "ACCESS2"]].describe()

,OBESITY,DIABETES,MHLTH,ACCESS2
count,83522.000000,83522.000000,83522.000000,83522.000000
mean,34.431157,12.429504,17.157563,11.478264
std,7.162231,3.794903,3.539844,7.465959
min,10.400000,0.700000,5.200000,1.300000
25%,29.600000,9.900000,14.700000,6.500000
50%,34.600000,12.000000,16.900000,9.300000
75%,39.200000,14.400000,19.200000,14.100000
max,64.400000,45.700000,42.800000,62.800000


In [12]:
state_summary = df_wide.groupby("StateAbbr")[["OBESITY", "DIABETES", "MHLTH", "ACCESS2"]].mean().round(2)

state_summary.head()

,OBESITY,DIABETES,MHLTH,ACCESS2
StateAbbr,,,,
AK,32.96,9.62,16.60,10.53
AL,40.52,15.58,19.15,11.82
AR,38.98,15.55,20.28,11.56
AZ,33.63,12.77,17.70,13.68
CA,28.57,11.68,16.67,11.17


In [13]:
df_wide[["OBESITY", "DIABETES", "MHLTH", "ACCESS2"]].corr()

,OBESITY,DIABETES,MHLTH,ACCESS2
OBESITY,1.000000,0.715865,0.647748,0.459638
DIABETES,0.715865,1.000000,0.420627,0.538574
MHLTH,0.647748,0.420627,1.000000,0.531974
ACCESS2,0.459638,0.538574,0.531974,1.000000
